# 4 - Removing Multicollinearity

In [106]:
import pandas as pd
pd.set_option('display.max_columns', None)
train = pd.read_pickle('../pickles/features/train_normalized')
test = pd.read_pickle('../pickles/features/test_normalized')

### Identify Collinearity<p> 
Per the contest page, I am going to use Lasso to identify and remove columns that create multicollinearity, starting with using GridSearch to find the best alpha value.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

train_x = train.drop('SalePrice',axis=1)
train_y = train['SalePrice']

In [109]:
lasso = Lasso()
params = {'alpha':[1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6, 1e7]}

In [110]:
search = GridSearchCV(lasso, params, scoring='neg_root_mean_squared_error', cv=5)

In [111]:
search.fit(train_x,train_y)

c:\Users\mebar\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.214e+11, tolerance: 6.882e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\mebar\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.452e+11, tolerance: 6.582e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\mebar\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.067e+11, toleranc

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   10.0, 100.0, 1000.0, 10000.0, 100000.0,
                                   1000000.0, 10000000.0]},
             scoring='neg_root_mean_squared_error')

In [112]:
search.best_estimator_

Lasso(alpha=100.0)

Now I run Lasso using that best estimator. 

In [113]:
search_best = search.best_estimator_

In [114]:
search_best.fit(train_x,train_y)

Lasso(alpha=100.0)

Save coefficients and corresponding column names

Now the resulting coefficients can be saved into a dataframe, along with the original column names and filtered to find any that have a **0** value. These are the ones that need to be removed.

In [115]:
coef = pd.Series(search_best.coef_,list(train_x.columns))
df = pd.DataFrame(data=coef)
df.head(6)

,0
LotFrontage,0.000000
LotArea,38020.163070
Street,-0.000000
Alley,-1010.485999
LotShape,1974.094127
LandContour,-441.626329


In [116]:
df[df[0]==0].head(5)

,0
LotFrontage,0.0
Street,-0.0
Utilities,0.0
BsmtFinSF2,0.0
BsmtUnfSF,0.0


In [124]:
display(df[df[0]==0].head(5))
print(f'Number of 0 coefficients: {len(df[df[0]==0].index.tolist())} \nNumber of columns checked: {train_x.shape[1]}')

,0
LotFrontage,0.0
Street,-0.0
Utilities,0.0
BsmtFinSF2,0.0
BsmtUnfSF,0.0


Number of 0 coefficients: 87 
Number of columns checked: 183


### Drop columns:

In [125]:
droplist = df[df[0]==0].index.tolist()

### Create new pickles: